<a href="https://colab.research.google.com/github/phycem/Data-Science-Projects-and-Tasks/blob/main/for_the_presentation_of_personal_project_Disaster_relief.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Starter libraries (double click to take a look) { display-mode: "form" }
# useful for opening files
import gdown
import zipfile

import os # accessing parts of your operating system
import re
import sys

# data visualization + manipulation -- we've seen these many times
import numpy as np
import pandas as pd

# plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt

# regression models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.preprocessing import PolynomialFeatures # for polynomial model
from sklearn.pipeline import Pipeline

# classification models
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# more sklearn model making
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

from collections import Counter

# NLP
import string
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords' ,quiet=True)
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TweetTokenizer
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import wordcloud
import tweepy

# NN models
import tensorflow as tf
import tensorflow_datasets as tfds

import keras
from tensorflow.keras import layers , activations , models , preprocessing, utils

# sequence data
from keras import Input, Model
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences

# text
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.text import Tokenizer

# NN and CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape
from keras.wrappers.scikit_learn import KerasClassifier
import keras.optimizers as optimizers
from keras.activations import softmax
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.applications import VGG16, VGG19, ResNet50, DenseNet121

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#@title Load your dataset { display-mode: "form" }
import pandas as pd
!wget -O ./disaster_data.csv 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Disaster%20Relief/disaster_data.csv'
dataset_path = './disaster_data.csv'
disaster_tweets = pd.read_csv('disaster_data.csv',encoding ="ISO-8859-1")

--2022-08-11 20:45:17--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Disaster%20Relief/disaster_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236777 (231K) [text/csv]
Saving to: ‘./disaster_data.csv’

./disaster_data.csv 100%[===================>] 231.23K  --.-KB/s    in 0.002s  

2022-08-11 20:45:17 (105 MB/s) - ‘./disaster_data.csv’ saved [236777/236777]



# Disaster Relief Project 
The data set for this project consists of tweets from people experiencing natural disasters around the world. The tweets are classified by category some are further classified by having some sort of need. 


In [ ]:
disaster_tweets

,tweet_id,text,category,need_or_resource
0,ca9e24c8-396d-4502-8b45-18895df5333e_0,"Donations of batteries, flashlights, and clean...",Energy,need
1,twitter_resource_tweets_1692,I want hurricane Sandy to cone so I can be stu...,None,NaN
2,625b46e2-0b81-41ea-826e-4535fe9b39b8,"Hi, I can help prepare food, serve food, offer...",Food,resource
3,twitter_resource_tweets_1699,I cant believe Sandy.....,None,NaN
4,c3bfea72-d377-445c-b4b8-e8ebca0e7fbb,I have children and adult clothes including ja...,Water,resource
...,...,...,...,...
1396,4954cd07-1796-4841-a79a-9edf7faf9521,We were here for the marathon and would like t...,Food,resource
1397,425e0f7e-eebc-4bc7-9027-34b3298ee084,I am writing on behalf of friends stranded in ...,Medical,need
1398,twitter_need_tweets_4630,Finally at my Uncle's house in Brooklyn. Gonna...,None,NaN
1399,twitter_resource_tweets_2745,Two years in a raw... #Irene then #Sandy... Co...,None,NaN


Differences between 80% test and 20% test


In [ ]:
#    help => no resource (mostly for offering help)    collect => food resource       tools => no resource    food => food resource    supplies => food need  power => energy need     batteries => energy resource
#                 => medical need                         => medical resource            =>no resource         => food Food_Resource      => medical resource          =>energy need            =>energy resource


At test %20 , Even tho there are more help words in other groups, the most accurance of "help" is at Medical Need which is the second highest needed. Other groups have it on third line. 

"collect" is not even listed at Medical resource or Food resource but is more accurate than  in other groups listed frequently.

"Food, tools, power and batteries" agrees with the plots 

Lastly, the cosine similarity function is able to spot which tweet group it belongs to.


---



In [ ]:
thegroups=[MedNeed,MedResource,WaterNeed,WaterResource,EnergyNeed, EnergyResource, FoodNeed,FoodResource,NoResource]

In [ ]:
print(vector_cosine_similarity(gets_vectors('Need a truck load of water (NOT DRINKING) to be able to start disinfecting home in Breezy Point.	'))) #true!

0.35398090833933876
0.6944013145057457
0.911626158405042
0.8427516392396618
0.7979364204653085
0.7617694250090519
0.7612716525270931
0.7707716484496777
0.706752322720476
None


In [ ]:
print(vector_cosine_similarity(gets_vectors('Basic tools, drill, etc.	'))) #true!

0.2783503257698568
0.49868193880889944
0.4045766289594256
0.48819219821452714
0.3779871047083524
0.5194717324159929
0.4063978629580659
0.4605490636767294
0.6410066899331396
None
